In [1]:
import yfinance as yf
import numpy as np

from gudhi.datasets.generators import points
from gudhi import SimplexTree

import plotly.graph_objects as go
from scipy.spatial.distance import cdist

In [ ]:
circle_sample = points.sphere(n_samples=500, ambient_dim=2, radius=1, sample="random")

fig = go.Figure(
  data=go.Scattergl(
    x=circle_sample[:,0],
    y=circle_sample[:,1],
    mode='markers'
  ),
  layout=go.Layout(
    width=1000,
    height=1000,
    paper_bgcolor="#FFFFFF",
    plot_bgcolor='#FFFFFF'
  )
)

fig.update_xaxes(showgrid=False, zeroline=False, range=[-2.,2.], tickvals=[])
fig.update_yaxes(showgrid=False, zeroline=False, range=[-2.,2.], tickvals=[])

fig.show()

In [21]:
point_array = points.sphere(n_samples=50, ambient_dim=3, radius=1, sample="random")
distance_matrix = cdist(point_array, point_array)
filtered_complex = SimplexTree.create_from_array(distance_matrix, max_filtration=0.5)

edge_array = np.array(
  [[point_array[i[0][0]],point_array[i[0][1]]] for i in filtered_complex.get_filtration() if len(i[0])>1]
)

filtration_array = np.array(
  [i[1] for i in filtered_complex.get_filtration() if len(i[0])>1]
)

fig = go.Figure(
  data=[go.Scatter3d(
    x=point_array[:,0],y=point_array[:,1],z=point_array[:,2],mode='markers', hoverinfo='none',marker=dict(size=5, color="#FFAD00"))],
  layout=go.Layout(
    width=1000,
    height=1000
  )
    )

for edge in edge_array:
  fig.add_trace(go.Scatter3d(x=edge[:,0],y=edge[:,1],z=edge[:,2],mode='lines',hoverinfo='none',line=dict(width=2.5,color="#0DE2FF")))

camera=dict(up=dict(x=0,y=0,z=1), center=dict(x=0,y=0,z=0), eye=dict(x=.15,y=.15,z=.15))

fig.update_layout(scene=dict(xaxis=dict(showbackground=True,showticklabels=False,title="",range=[-10,10],backgroundcolor="#0f1117",gridcolor='white'), 
                             yaxis=dict(showbackground=True,showticklabels=False,title="",range=[-10,10],backgroundcolor="#0f1117",gridcolor='white'), 
                             zaxis=dict(showbackground=True,showticklabels=False,title="",range=[-10,10],backgroundcolor="#0f1117",gridcolor='white'),
                             camera=camera), 
                  margin=dict(l=0, r=0, b=0, t=0),
                  showlegend=False
                  )
fig.show()